In [51]:
import os
import time
import sys
import numpy as np
import vectorbt as vbt
import pandas as pd

In [79]:
price = vbt.YFData.download('BTC-USD', start='2018-01-01', end = '2024-01-01')
price

In [87]:
st = time.time()
close = price.get('Close')
fast_ma = vbt.MA.run(close, 5, short_name='fast_ma')
slow_ma = vbt.MA.run(close, 10, short_name='slow_ma')
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)
pf = vbt.Portfolio.from_signals(close, entries, exits, fees=0.000)
et = time.time()
print(f'Elapsed time: {1000*(et-st):.2f} ms')
print(f'Total return: {pf.total_return()}')

Elapsed time: 38.49 ms
Total return: 2.09876466123077


In [83]:
#pf.orders.records_readable

pf.plot_cum_returns()


FigureWidget({
    'data': [{'line': {'color': '#7f7f7f'},
              'name': 'Benchmark',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'b7a16da6-3135-4e64-9209-bd76dbc9d617',
              'x': array([datetime.datetime(2018, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2018, 1, 2, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2018, 1, 3, 0, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2023, 12, 30, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 12, 31, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 1, 1, 0, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([1.        , 1.09701106, 1.11303926, ..., 3.08678951, 3.09471831,
                          3.23399609])},
             {'hoverinfo': 'skip',
   

In [89]:
symbols = ["BTC-USD"]
price = vbt.YFData.download(
    symbols, 
    start='2018-01-01',
    end = '2024-01-01',
    missing_index='drop').get('Close')
price.shape

(2192,)

In [136]:
st = time.time()
windows = np.arange(5, 11)
fast_ma, slow_ma = vbt.MA.run_combs(price, window=windows, r=2, short_names=['fast', 'slow'])
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

pf_kwargs = dict(size=np.inf, fees=0.000, freq='1D')
pf = vbt.Portfolio.from_signals(price, entries, exits, **pf_kwargs)
et = time.time()
print(f'Elapsed time: {1000*(et-st):.2f} ms')

Elapsed time: 55.98 ms


In [137]:
np.mean(pf.total_return().values)

2.9724106456555077

In [138]:
pf.total_return()

fast_window  slow_window
5            6              2.095954
             7              1.647425
             8              4.704819
             9              2.629303
             10             2.098765
             11             2.507180
             12             2.468184
6            7              4.468970
             8              5.106150
             9              3.552162
             10             2.211441
             11             2.216959
             12             2.690064
7            8              7.707811
             9              1.355910
             10             2.236120
             11             1.633518
             12             2.774546
8            9              0.233879
             10             1.820629
             11             3.630939
             12             2.481364
9            10             3.521910
             11             6.969953
             12             2.732578
10           11             4.286548
             